### [Spark Developer Certification - Comprehensive Study Guide](https://github.com/mdrakiburrahman/databricks-certification)

~/spark/databrick-cert/databricks-certification/Comprehensive_study_guide_for_Spark_Developer_Certification.html

### Tips and Tricks

#### Driver

The driver is the machine in which the application runs (master node). It is responsible for three main things: 
* Maintaining information about the Spark Application, 
* Responding to the user’s program, 
* Analyzing, distributing, and scheduling work across the executors.

Worker nodes performs computation in parallel.


#### Dynamic Partition Pruning (DPP) 

DPP can auto-optimize your queries and make them more performant automatically.
enabling it via property `spark.sql.optimizer.dynamicPartitionPruning.enabled`

#### SkewJoin

Spark dynamically handles skew in sort-merge join by splitting (and replicating if needed) skewed partitions. enabling it via property `spark.sql.adaptive.skewJoin.enabled`

#### cache

When you use cache() or persist(), the DataFrame is not fully cached until you invoke an action that goes through every record (e.g., count()). If you use an action like take(1), only one partition will be cached because Catalyst realizes that you do not need to compute all the partitions just to retrieve one record.


When using DataFrame.persist() data on disk is always serialized.

#### broadcast

By default `spark.sql.autoBroadcastJoinThreshold = 10MB`, any value above this threashold will not force a broadcast join

#### dataframe sort

`df.orderBy(desc_nulls_first("a"))` will sort a df with column "a" containing null and keep null first. (similarly `desc_nulls_last("a")`).  
* `df.sort(*cols, **kwargs)` is another sort API.
* more examples - https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.sort.html#pyspark.sql.DataFrame.sort

#### join-expression
pyspark uses `==` but scalar uses `===`

#### JVM garbage collection
remember that the cost of garbage collection is proportional to the number of Java objects. To reduce `gc`, 
* create fewer objects
* increase java heap space size
* persist objects in serialized form

#### global external/unmanaged table

Spark manages the metadata, while you control the data location. As soon as you add `path` option in dataframe writer it will be treated as global external/unmanaged table. When you drop table only metadata gets dropped. A global unmanaged/external table is available across all clusters.

When defining a table from files on disk, you create an unmanaged table.
When you use `saveAsTable` on a dataframe, you create a managed table for which Spark with track both metadata and data.

#### execution mode vs deployment mode

3 execution modes:
* cluster
* client
* local

4 deployment modes:
* local - driver/executors in one machine (non-cluster mode)
* standalone - spark cluster runs only spark apps 
* YARN - spark app co-exists with other non-spark JVM apps
* Meso - schedule JVM, C++, python apps, manage CPU/Net resources plus Memory

#### Hive data_type to define Spark DataFrame schema
- https://cwiki.apache.org/confluence/display/Hive/LanguageManual+Types

* INT, SMALLINT, TINYINT, BIGINT
* FLOAT, DOUBLE, DECIMAL, NUMERIC
* DATE, TIMESTAMP, INTERVAL
* STRING, VARCHAR, CHAR
* BOOLEAN
* BINARY


#### Review Spark conf settings
https://spark.apache.org/docs/latest/configuration.html

- Spark properties control most application parameters and can be set by using a SparkConf object, or through Java system properties.
- Environment variables can be used to set per-machine settings, such as the IP address, through the conf/spark-env.sh script on each node.
- Logging can be configured through log4j.properties.
    
    
#### Cluster Overview
https://spark.apache.org/docs/latest/cluster-overview.html

#### Repartition vs Coalesce
https://ashwin.cloud/blog/spark-repartition-vs-coalesce/

Repartition can be used for increasing or decreasing the number of partitions. Whereas Coalesce can only be used for decreasing the number of partitions. Coalesce is a less expensive operation than Repartition as Coalesce reduces data movement between the nodes while Repartition shuffles all data over the network.


#### Structured streaming guide
https://spark.apache.org/docs/3.1.1/structured-streaming-programming-guide.html#schema-inference-and-partition-of-streaming-dataframesdatasets

In [1]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import *
from pyspark import StorageLevel
import sys

In [2]:
from IPython.display import display

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
spark

In [5]:
list_df = spark.createDataFrame([1, 2, 3, 4], IntegerType())

In [6]:
list_df.show()

+-----+
|value|
+-----+
|    1|
|    2|
|    3|
|    4|
+-----+



In [7]:
# Create Example Data - Departments and Employees

# Create the Employees
Employee = Row("name","gender") # Define the Row `Employee' with one column/key
employee1 = Employee('Bob',"M") # Define against the Row 'Employee'
employee2 = Employee('Sam',"M") # Define against the Row 'Employee'
employee3 = Employee('Jane',"F") # Define against the Row 'Employee'

# Create the Departments
Department = Row("name", "department") # Define the Row `Department' with two columns/keys
department1 = Department('Bob', 'Accounts') # Define against the Row 'Department'
department2 = Department('Alice', 'Sales') # Define against the Row 'Department'
department3 = Department('Sam', 'HR') # Define against the Row 'Department'

# Create DataFrames from rows
employeeDF = spark.createDataFrame([employee1, employee2]) 
departmentDF = spark.createDataFrame([department1, department2, department3])

In [8]:
joinExpression = employeeDF["name"] == departmentDF["name"]

In [9]:
employeeDF.join(departmentDF, joinExpression, how="inner").show()

+----+------+----+----------+
|name|gender|name|department|
+----+------+----+----------+
| Bob|     M| Bob|  Accounts|
| Sam|     M| Sam|        HR|
+----+------+----+----------+



In [10]:
employeeDF.join(departmentDF, joinExpression, how="left_outer").show()

+----+------+----+----------+
|name|gender|name|department|
+----+------+----+----------+
| Bob|     M| Bob|  Accounts|
| Sam|     M| Sam|        HR|
+----+------+----+----------+



In [11]:
employeeDF.join(departmentDF, joinExpression, how="left_semi").show()

+----+------+
|name|gender|
+----+------+
| Bob|     M|
| Sam|     M|
+----+------+



In [12]:
employeeDF.join(departmentDF, joinExpression, how="left_anti").show()

+----+------+
|name|gender|
+----+------+
+----+------+



In [35]:
employeeDF.join(departmentDF, joinExpression, how="right_outer").show()

+----+------+-----+----------+
|name|gender| name|department|
+----+------+-----+----------+
| Bob|     M|  Bob|  Accounts|
| Sam|     M|  Sam|        HR|
|null|  null|Alice|     Sales|
+----+------+-----+----------+



In [36]:
employeeDF.join(departmentDF, joinExpression, how="cross").show()

+----+------+----+----------+
|name|gender|name|department|
+----+------+----+----------+
| Bob|     M| Bob|  Accounts|
| Sam|     M| Sam|        HR|
+----+------+----+----------+



In [37]:
employeeDF.join(departmentDF, joinExpression, how="outer").show()

+----+------+-----+----------+
|name|gender| name|department|
+----+------+-----+----------+
| Bob|     M|  Bob|  Accounts|
| Sam|     M|  Sam|        HR|
|null|  null|Alice|     Sales|
+----+------+-----+----------+



In [39]:
employeeDF.join(departmentDF, joinExpression, how="full").show()

+----+------+-----+----------+
|name|gender| name|department|
+----+------+-----+----------+
| Bob|     M|  Bob|  Accounts|
| Sam|     M|  Sam|        HR|
|null|  null|Alice|     Sales|
+----+------+-----+----------+



In [13]:
schema = StructType(
    [
        StructField("letter", StringType(), True),
        StructField("position", IntegerType(), True),
    ]
)

data = [("A", 1), ("B", 2), ("C", 3)]
df = spark.createDataFrame(data=data, schema=schema)
df.show()

+------+--------+
|letter|position|
+------+--------+
|     A|       1|
|     B|       2|
|     C|       3|
+------+--------+



In [15]:
schema = df.schema

In [17]:
schema.fieldNames()

['letter', 'position']

In [18]:
schema.add("value",IntegerType(),True)

StructType(List(StructField(letter,StringType,true),StructField(position,IntegerType,true),StructField(value,IntegerType,true)))

In [19]:
schema.fieldNames()

['letter', 'position', 'value']

In [20]:
df.show()

+------+--------+
|letter|position|
+------+--------+
|     A|       1|
|     B|       2|
|     C|       3|
+------+--------+



In [22]:
df = df.withColumn("value", lit(100))
df.show()

+------+--------+-----+
|letter|position|value|
+------+--------+-----+
|     A|       1|  100|
|     B|       2|  100|
|     C|       3|  100|
+------+--------+-----+



In [23]:
df.schema.fieldNames()

['letter', 'position', 'value']

In [24]:
df.columns

['letter', 'position', 'value']

In [33]:
# add new rows with union
df = df.union(spark.createDataFrame(data=[["D", 4, 1000]], schema=df.schema))

In [32]:
df.show()

+------+--------+-----+
|letter|position|value|
+------+--------+-----+
|     A|       1|  100|
|     B|       2|  100|
|     C|       3|  100|
|     D|       4| 1000|
+------+--------+-----+



In [27]:
df.groupBy("letter").sum("value").collect()[0]

Row(letter='B', sum(value)=100)

In [29]:
df.agg({"value":"sum", "letter":"count"}).show()

+-------------+----------+
|count(letter)|sum(value)|
+-------------+----------+
|            3|       300|
+-------------+----------+



In [41]:
# Create Example Data - Departments and Employees

# Create the Departments
Department = Row("id", "name")
department1 = Department('123456', 'Computer Science')
department2 = Department('789012', 'Mechanical Engineering')
department3 = Department('345678', 'Theater and Drama')
department4 = Department('901234', 'Indoor Recreation')
department5 = Department('000000', 'All Students')

# Create the Employees
Employee = Row("firstName", "lastName", "email", "salary")
employee1 = Employee('michael', 'armbrust', 'no-reply@berkeley.edu', 100000)
employee2 = Employee('xiangrui', 'meng', 'no-reply@stanford.edu', 120000)
employee3 = Employee('matei', None, 'no-reply@waterloo.edu', 140000)
employee4 = Employee(None, 'wendell', 'no-reply@berkeley.edu', 160000)
employee5 = Employee('michael', 'jackson', 'no-reply@neverla.nd', 80000)

# Create the DepartmentWithEmployees instances from Departments and Employees
DepartmentWithEmployees = Row("department", "employees")
departmentWithEmployees1 = DepartmentWithEmployees(department1, [employee1, employee2])
departmentWithEmployees2 = DepartmentWithEmployees(department2, [employee3, employee4])
departmentWithEmployees3 = DepartmentWithEmployees(department3, [employee5, employee4])
departmentWithEmployees4 = DepartmentWithEmployees(department4, [employee2, employee3])
departmentWithEmployees5 = DepartmentWithEmployees(department5, [employee1, employee2, employee3, employee4, employee5])

print(department1)
print(employee2)
print(departmentWithEmployees1.employees[0].email)

Row(id='123456', name='Computer Science')
Row(firstName='xiangrui', lastName='meng', email='no-reply@stanford.edu', salary=120000)
no-reply@berkeley.edu


In [43]:
departmentsWithEmployeesSeq1 = [departmentWithEmployees1, departmentWithEmployees2, departmentWithEmployees3, departmentWithEmployees4, departmentWithEmployees5]
df1 = spark.createDataFrame(departmentsWithEmployeesSeq1)
df1.show(truncate=False)

+--------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|department                      |employees                                                                                                                                                                                                                                 |
+--------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[123456, Computer Science]      |[[michael, armbrust, no-reply@berkeley.edu, 100000], [xiangrui, meng, no-reply@stanford.edu, 120000]]                                                       

In [45]:
df1.show(vertical=True, truncate=False)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 department | [123456, Computer Science]                                                                                                                                                                                                                 
 employees  | [[michael, armbrust, no-reply@berkeley.edu, 100000], [xiangrui, meng, no-reply@stanford.edu, 120000]]                                                                                                                                      
-RECORD 1------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [46]:
df = spark.range(1,8,2).toDF("number")
df.show()

+------+
|number|
+------+
|     1|
|     3|
|     5|
|     7|
+------+



### sample question for certification

How to create spark dataframe from list

https://stackoverflow.com/questions/43444925/how-to-create-dataframe-from-list-in-spark-sql/50969995

#### Q1

In [55]:
schema = StructType([
                StructField("Name", StringType())
               ,StructField("Age", IntegerType())
               ,StructField("Score", DoubleType())
               ,StructField("DOB", StringType())
              ])

# using HIVE DDL
# schema = "Name STRING, Age INT, Score DOUBLE, DOB VARCHAR(10)" 

data = [
    ('Alice', 20, 700.50, "2001-10-02"), 
    ('Bob', 15, 500.50, "2006-01-01"), 
]

df2 = spark.createDataFrame(data=data, schema=schema) 
df2.show()

+-----+---+-----+----------+
| Name|Age|Score|       DOB|
+-----+---+-----+----------+
|Alice| 20|700.5|2001-10-02|
|  Bob| 15|500.5|2006-01-01|
+-----+---+-----+----------+



#### Q2

In [57]:
a = [1002, 3001, 4002, 2003, 2002, 3004, 1003, 4006]
b = (spark
  .createDataFrame(list(map(lambda x: Row(value=x), a)))
  .withColumn("x", col("value") % 1000)
)

In [60]:
b.sort(col("x")).show()

+-----+---+
|value|  x|
+-----+---+
| 3001|  1|
| 4002|  2|
| 1002|  2|
| 2002|  2|
| 1003|  3|
| 2003|  3|
| 3004|  4|
| 4006|  6|
+-----+---+



In [61]:
c = (
    b.groupBy(col("x"))
    .agg(count("x"), sum("value"))
    .drop("x")
    .toDF("count", "total")
    .orderBy(col("count").desc(), col("total"))
    .limit(1)
    .show()
)

+-----+-----+
|count|total|
+-----+-----+
|    3| 7006|
+-----+-----+



#### Q3

In [63]:
data_schema = StructType(
    [
        StructField("UserKey", IntegerType())
        ,StructField("ItemKey", IntegerType())
        ,StructField("ItemName", StringType())
        ,StructField("Score", FloatType())
    ]
)

data_list = [
  (1, 1000, "Apple", 0.76),
  (2, 1000, "Apple", 0.11),
  (1, 2000, "Orange", 0.98),
  (1, 3000, "Banana", 0.24),
  (2, 3000, "Banana", 0.99)    
]

data_df = spark.createDataFrame(data_list, schema=data_schema) 
data_df.show()

+-------+-------+--------+-----+
|UserKey|ItemKey|ItemName|Score|
+-------+-------+--------+-----+
|      1|   1000|   Apple| 0.76|
|      2|   1000|   Apple| 0.11|
|      1|   2000|  Orange| 0.98|
|      1|   3000|  Banana| 0.24|
|      2|   3000|  Banana| 0.99|
+-------+-------+--------+-----+



In [65]:
(
data_df.groupBy("UserKey")
  .agg(sort_array(collect_list(struct("Score", "ItemKey", "ItemName")), asc=False))
  .toDF("UserKey", "Collection")
  .show(20, False)
)

+-------+-----------------------------------------------------------------+
|UserKey|Collection                                                       |
+-------+-----------------------------------------------------------------+
|1      |[[0.98, 2000, Orange], [0.76, 1000, Apple], [0.24, 3000, Banana]]|
|2      |[[0.99, 3000, Banana], [0.11, 1000, Apple]]                      |
+-------+-----------------------------------------------------------------+



#### Q4 window

In [66]:
people_schema = StructType([
                  StructField("name", StringType())
                 ,StructField("department", IntegerType())
                 ,StructField("score", ArrayType(IntegerType()))
              ])

people_list = [
    ("Ali", 0, [100]),
    ("Barbara", 1, [300, 250, 100]),
    ("Cesar", 1, [350, 100]),
    ("Dongmei", 1, [400, 100]),
    ("Eli", 2, [250]),
    ("Florita", 2, [500, 300, 100]),
    ("Gatimu", 3, [300, 100])
]


people_df = spark.createDataFrame(people_list, schema=people_schema) 
people_df.show()

+-------+----------+---------------+
|   name|department|          score|
+-------+----------+---------------+
|    Ali|         0|          [100]|
|Barbara|         1|[300, 250, 100]|
|  Cesar|         1|     [350, 100]|
|Dongmei|         1|     [400, 100]|
|    Eli|         2|          [250]|
|Florita|         2|[500, 300, 100]|
| Gatimu|         3|     [300, 100]|
+-------+----------+---------------+



In [68]:
from pyspark.sql.window import Window
from pyspark.sql.functions import explode, dense_rank, max

windowSpec = Window.partitionBy("department").orderBy(col("score").desc())

In [69]:
# look at intermediate result
(
people_df
  .withColumn("score", explode(col("score")))
  .select(
    col("department"),
    col("name"),
    col("score"),
    dense_rank().over(windowSpec).alias("rank"),
    max(col("score")).over(windowSpec).alias("highest")
  )
  .show()
)

+----------+-------+-----+----+-------+
|department|   name|score|rank|highest|
+----------+-------+-----+----+-------+
|         1|Dongmei|  400|   1|    400|
|         1|  Cesar|  350|   2|    400|
|         1|Barbara|  300|   3|    400|
|         1|Barbara|  250|   4|    400|
|         1|Barbara|  100|   5|    400|
|         1|  Cesar|  100|   5|    400|
|         1|Dongmei|  100|   5|    400|
|         3| Gatimu|  300|   1|    300|
|         3| Gatimu|  100|   2|    300|
|         2|Florita|  500|   1|    500|
|         2|Florita|  300|   2|    500|
|         2|    Eli|  250|   3|    500|
|         2|Florita|  100|   4|    500|
|         0|    Ali|  100|   1|    100|
+----------+-------+-----+----+-------+



In [74]:
(
people_df
  .withColumn("score", explode(col("score")))
  .select(
    col("department"),
    col("name"),
    dense_rank().over(windowSpec).alias("rank"),
    max(col("score")).over(windowSpec).alias("highest")
  )
  .where(col("rank") == 1)
  .drop("rank")
  .orderBy("department")
  .show()
)

+----------+-------+-------+
|department|   name|highest|
+----------+-------+-------+
|         0|    Ali|    100|
|         1|Dongmei|    400|
|         2|Florita|    500|
|         3| Gatimu|    300|
+----------+-------+-------+

